In [6]:
import os
import json
import time
import numpy as np
import pandas as pd

from tqdm.auto import tqdm
from collections import defaultdict
from collections import Counter

import fasttext
import faiss

from tqdm.auto import tqdm

In [3]:
data_path = '../data/train_sessions.jsonl'
  
train_sessions = pd.DataFrame()
chunks = pd.read_json(data_path, lines=True, chunksize=100_000)

for e, chunk in enumerate(chunks):
    event_dict = {
        'session': [],
        'aid': [],
        'ts': [],
        'type': [],
    }
    if e < 2:
        # train_sessions = pd.concat([train_sessions, chunk])
        for session, events in zip(chunk['session'].tolist(), chunk['events'].tolist()):
            for event in events:
                event_dict['session'].append(session)
                event_dict['aid'].append(event['aid'])
                event_dict['ts'].append(event['ts'])
                event_dict['type'].append(event['type'])
        chunk_session = pd.DataFrame(event_dict)
        train_sessions = pd.concat([train_sessions, chunk_session])
    else:
        break
        
train_sessions = train_sessions.reset_index(drop=True)
train_sessions.head()

,session,aid,ts,type
0,0,1517085,1659304800025,clicks
1,0,1563459,1659304904511,clicks
2,0,1309446,1659367439426,clicks
3,0,16246,1659367719997,clicks
4,0,1781822,1659367871344,clicks


## Hot

In [20]:
hot_df = train_sessions.copy()
hot_df["score"] = hot_df.type.replace(["clicks", "carts", "orders"], [1, 2, 3])
hot_df = hot_df.groupby(["aid"]).sum()["score"].sort_values(ascending=False).reset_index()
hot_df.head()

,aid,score
0,29735,6727
1,1603001,6349
2,1733943,6232
3,80222,6079
4,832192,5651


## Item2Vec

In [4]:
aid_seq = train_sessions.sort_values(["session", "ts"]).reset_index(drop=True)
aid_seq["aid_2"] = aid_seq.aid.shift(1)
aid_seq = aid_seq[aid_seq.aid != aid_seq.aid_2]
aid_seq = aid_seq[["session", "aid", "ts", "type"]]
aid_seq["aid"] = aid_seq["aid"].astype(str)
aid_seq = aid_seq.groupby(["session"]).agg(list)["aid"].reset_index()
aid_seq = aid_seq[(aid_seq.aid.apply(len) >= 5)].reset_index(drop=True)
aid_seq.head()

,session,aid
0,0,"[1517085, 1563459, 1309446, 16246, 1781822, 11..."
1,1,"[424964, 1492293, 910862, 1491172, 424964, 151..."
2,2,"[763743, 137492, 504789, 137492, 795863, 37834..."
3,3,"[1425967, 1343406, 1425967, 1343406, 1815570, ..."
4,4,"[613619, 298827, 383828, 255379, 1838173, 1453..."


In [7]:
with open("../data/train_aid_seq.txt", 'w') as f:
    for aid_list in aid_seq.aid:
        print("__label__1", " ".join(aid_list), file = f)
        
model = fasttext.train_unsupervised(
    '../data/train_aid_seq.txt', 
    model = 'skipgram',
    ws = 5,
    dim = 128, 
    epoch = 5, 
    lr = 0.01,
    minn = 0,
    maxn = 0
)

Read 8M words
Number of words:  275012
Number of labels: 1
Progress: 100.0% words/sec/thread:   62271 lr:  0.000000 avg.loss:  2.700827 ETA:   0h 0m 0s


In [8]:
item_dict = {}
embeddings = []
for i, item_id in enumerate(model.words):
    if item_id != '</s>':
        item_dict[i-1] = item_id
        embeddings.append(model[item_id])
    
index_data = np.array(embeddings)
faiss.normalize_L2(index_data)
index = faiss.IndexHNSWFlat(128, 32, faiss.METRIC_INNER_PRODUCT)

index.add(index_data)
sim_matrix, candidate_ids_matrix = index.search(index_data, 20)

sim_index = []
for idx, candidate_ids in tqdm(enumerate(candidate_ids_matrix)):
    sims = sim_matrix[idx]
    candidates = list(zip(candidate_ids, sims))
    candidates = list(filter(lambda x: x[0] != idx, candidates))
    candidates = list(filter(lambda x: x[0] in item_dict, candidates))
    sim_index.extend([(item_dict[idx], item_dict[x[0]], float(x[1])) for x in candidates])

0it [00:00, ?it/s]